<a href="https://colab.research.google.com/github/Kensuzuki95/EY_Assignment/blob/main/EY_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Packages

In [93]:
import os
import pandas as pd
import xlrd
import openpyxl
import glob
import shutil
!pip install pandas pyxlsb

# Investigate Folder Content

In [58]:
# folder path
dir_path = r'/content/drive/MyDrive/EY Recruitment Documents/invoice'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 4406


In [51]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice',"*.xlsx"))
tifCounter

4400

In [73]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice',"*.xlsb"))
tifCounter

6

## Sort Files depending on Extension Types

In [76]:
def extract_files_by_extension(source_folder, destination_folder, extension):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate over files in the source folder
    for filename in os.listdir(source_folder):
        if filename.endswith(extension):
            # Get the full path of the source file
            source_file = os.path.join(source_folder, filename)

            # Get the full path of the destination file
            destination_file = os.path.join(destination_folder, filename)

            # Copy the file to the destination folder
            shutil.copy2(source_file, destination_file)

### Folder with .xlsx extension Invoices

In [77]:
# Create a folder with only .xlsx files
source_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
destination_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsx'
extension = '.xlsx'

extract_files_by_extension(source_folder, destination_folder, extension)

In [103]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsx',"*.xlsx"))
tifCounter

4400

### Folder with .xlsb extention Invoices

In [78]:
# Create a folder with only .xlsb files
source_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
destination_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb'
extension = '.xlsb'

extract_files_by_extension(source_folder, destination_folder, extension)

In [104]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb',"*.xlsb"))
tifCounter

6

In [110]:
def convert_xlsb_to_xlsx(input_folder, output_folder):
    # Get a list of all files in the input folder
    files = os.listdir(input_folder)

    # Filter out only the .xlsb files
    xlsb_files = [file for file in files if file.endswith('.xlsb')]

    # Iterate over the .xlsb files and convert them to .xlsx
    for file in xlsb_files:
        # Construct the input file path
        input_file_path = os.path.join(input_folder, file)

        # Construct the output file path by replacing the extension and adding the output folder
        output_file_path = os.path.join(output_folder, os.path.splitext(file)[0] + '.xlsx')

        # Read the .xlsb file using pandas
        df = pd.read_excel(input_file_path, engine='pyxlsb')

        # Write the DataFrame to the .xlsx file
        df.to_excel(output_file_path, index=False)

        print(f"Converted {input_file_path} to {output_file_path}")

In [111]:
# Provide the input and output folder paths
input_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb'
output_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

convert_xlsb_to_xlsx(input_folder, output_folder)

Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/202003000008681.xlsb to /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx/202003000008681.xlsx
Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/202004000002111.xlsb to /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx/202004000002111.xlsx
Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/202010000002111.xlsb to /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx/202010000002111.xlsx
Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/202001000002111.xlsb to /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx/202001000002111.xlsx
Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/202006000008681.xlsb to /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx/202006000008681.xlsx
Converted /content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb/2020

#Extract Data from Files

In [105]:
folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
output_file = '/content/drive/MyDrive/EY Recruitment Documents/invoice_data.xlsx'

In [99]:
output_wb = openpyxl.Workbook()
output_sheet = output_wb.active
output_sheet.title = 'Invoice Data'

In [100]:
cells = ['A2', 'G2', 'G3', 'A15', 'D15', 'A16', 'D16', 'A17', 'D17', 'A18', 'D18', 'A19', 'D19']

In [101]:
for filename in os.listdir(folder):
  if filename.endswith('.xlsx'):
    file = os.path.join(folder, filename)
    workbook = openpyxl.load_workbook(file)
    values = [workbook.active[cell].value for cell in cells]
    output_sheet.append(values)

output_wb.save(output_file)

In [108]:
dataset = pd.read_excel(
    '/content/drive/MyDrive/EY Recruitment Documents/invoice_data.xlsx'
    ).replace(r'^s*$', float('NaN'), regex = True)
dataset.head()
dataset.info()
#len(dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4399 entries, 0 to 4398
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4313 non-null   object 
 1   Unnamed: 1  4313 non-null   float64
 2   Unnamed: 2  4313 non-null   object 
 3   Unnamed: 3  4313 non-null   object 
 4   Unnamed: 4  4313 non-null   float64
 5   Unnamed: 5  4313 non-null   object 
 6   Unnamed: 6  4313 non-null   float64
 7   Unnamed: 7  970 non-null    object 
 8   Unnamed: 8  970 non-null    float64
dtypes: float64(4), object(5)
memory usage: 309.4+ KB


In [ ]:
# Add empty row on top
#df1 = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns)
#df = df1.append(df, ignore_index=True)

# Notes

In [ ]:
pd.read_excel('/content/drive/MyDrive/EY Recruitment Documents/202304000109881.xlsx')

In [ ]:
pd.read_csv('/content/drive/MyDrive/EY Recruitment Documents/202001000005241.xlsx - invoice.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/EY Recruitment Documents/202304000109881.xlsx - 請求書.csv')